## Integração de dados

#### Masterlens, CASTLES

In [241]:
import pandas as pd
from astropy.coordinates import Angle

In [242]:
def deg_to_arcsec(coord):
    return coord*3600


In [243]:
def hour_to_arcsec(coord):  # string coord
    result = Angle(coord).degree
    result = result*3600
    return result

In [244]:
def hour(coord):
    result = coord.replace(":", 'h', 1)
    result = result.replace(":", 'm', 1)
    result += 's'
    return result

In [245]:
# lens as a whole
castles = pd.read_csv('Scraping/castles.csv')
castles.drop(['zs', 'zl', 'ms (mag)', 'ml (mag)', 'dt (days)'], axis=1, inplace=True)
castles.head()

,Lens Name,RA (J2000),Dec (J2000),E(B-V),FGHz (mJy),Nim,"size ("")",sigma (km/s)
0,Q0047-2808,00:49:41.89,-27:52:25.7,0.016,NaN,4ER,2.70,229±15
1,HE0047-1756,00:50:27.83,-17:40:8.8,0.022,NaN,2,1.44,NaN
2,HST01247+0352,01:24:44.4,+03:52:00,0.029,NaN,2,2.20,NaN
3,HST01248+0351,01:24:45.6,+03:51:06,0.029,NaN,2,0.74,NaN
4,B0128+437,01:31:13.405,+43:58:13.14,0.082,F5=48,4,0.55,NaN


In [246]:
castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(hour)
castles.head()

,Lens Name,RA (J2000),Dec (J2000),E(B-V),FGHz (mJy),Nim,"size ("")",sigma (km/s)
0,Q0047-2808,00h49m41.89s,-27h52m25.7s,0.016,NaN,4ER,2.70,229±15
1,HE0047-1756,00h50m27.83s,-17h40m8.8s,0.022,NaN,2,1.44,NaN
2,HST01247+0352,01h24m44.4s,+03h52m00s,0.029,NaN,2,2.20,NaN
3,HST01248+0351,01h24m45.6s,+03h51m06s,0.029,NaN,2,0.74,NaN
4,B0128+437,01h31m13.405s,+43h58m13.14s,0.082,F5=48,4,0.55,NaN


In [247]:
castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_arcsec)
#castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(hour_to_arcsec)
castles.head()

,Lens Name,RA (J2000),Dec (J2000),E(B-V),FGHz (mJy),Nim,"size ("")",sigma (km/s)
0,Q0047-2808,44728.350,-27h52m25.7s,0.016,NaN,4ER,2.70,229±15
1,HE0047-1756,45417.450,-17h40m8.8s,0.022,NaN,2,1.44,NaN
2,HST01247+0352,76266.000,+03h52m00s,0.029,NaN,2,2.20,NaN
3,HST01248+0351,76284.000,+03h51m06s,0.029,NaN,2,0.74,NaN
4,B0128+437,82101.075,+43h58m13.14s,0.082,F5=48,4,0.55,NaN


In [4]:
# lens objects  -  227
castles2 = pd.read_csv('CastelLensData.csv')
castles2.head()

,lens_names,ra,ra_err,dec,dec_err,description
0,B0218+357,2.35152,NaN,35.9372,NaN,source
1,B0218+357,2.65852,0.003,36.0632,0.003,deflected
2,B0218+357,2.52952,0.011,35.9832,0.024,galaxy
3,B0712+472,6.47266,0.003,46.9912,0.003,deflected
4,B0712+472,6.52066,0.003,46.8552,0.006,deflected


In [249]:
def system():  # lens as a whole
    
    # importing castles data
    castles = pd.read_csv('CastelInfo.csv')
    castles = castles.drop(['z_source', 'z_lens'], axis=1)
    castles.columns=['Name', 'N Images_c', 'RA arcsec_c', 'Dec arcsec_c']  # change
    castles[['RA arcsec_c', 'Dec arcsec_c']] = castles[['RA arcsec_c', 'Dec arcsec_c']].apply(deg_to_arcsec)
    castles = castles.set_index('Name')
    
    dfs = []
    for i in range(1,51):
          
        # importing masterlens data
        ml = pd.read_csv('Scraping/results/system_{}.csv'.format(i))  # getting data from system file
        for co in ['z_lens', 'z_lens_err', 'z_source', 'z_source_err', 'z_Lens quality', 'z_Source quality',
                   'RA Hrs', 'RA Mins', 'RA Secs', 'Dec Degrees', 'Dec Arcmin', 'Dec Arcsec', 'RA [°]', 'Dec [°]',
                  'Coordinates:', 'Discovery.1']:
            if co in list(ml.columns):
                ml = ml.drop(co, axis=1)
        
        coords = pd.read_csv('Scraping/results/coordinates_{}.csv'.format(i)) # getting coords from coords file
        ml['Ra_arcsec_ml'] = float(coords['RA [°]'].apply(deg_to_arcsec))
        ml['Dec_arcsec_ml'] = coords['Dec [°]'].apply(deg_to_arcsec)
        
        #coords = coords[['RA [°]', 'Dec [°]']]
        
        # joining sys + coords data and converting coords to arcsec
        #ml[['RA arcsec', 'Dec arcsec']] = coords.apply(deg_to_arcsec)
   
        dfs.append(ml)
            
    all_ml = pd.concat(dfs, sort=False) # joining all masterlens system dfs
    all_ml = all_ml.set_index('Name')

    # joining castles + ml data
    result = pd.merge(castles, all_ml, how ='outer', on='Name')  

    # full outer join (castles + ml)
    
    #print(all_ml.index)
    #result = pd.concat([castles, all_ml], axis=1).reindex(castles.index)
    return result
        

In [250]:
system_df = system()
system_df #### z_Lens?

,N Images_c,RA arcsec_c,Dec arcsec_c,Discovery,Lens Kind,Lens Grade,Einstein_R quality,Description,Einstein_R,Einstein_R_err,Stellar_v_disp,Stellar_v_disp_err,Discovery Date,N Images,Ra_arcsec_ml,Dec_arcsec_ml,Reference redshift,z_Lens,z_Source(s),"Einstein_R [""]"
Name,,,,,,,,,,,,,,,,,,,,
B0218+357,2ER,8465.4720,129373.9200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0712+472,4,26163.5760,169729.9200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B1030+074,2,38014.2000,25885.5120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B1152+200,2,42918.1200,70782.1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B1422+231,4E,51878.1600,82560.6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B1600+434,2,57700.4400,155807.6400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B1608+656,4,58154.0400,235949.0400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B1933+503,10,70471.0800,181523.5200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B1938+666,R,70705.0800,240532.2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# junção parcial 

In [96]:
ml = pd.read_csv('Scraping/results/system_1.csv')  # getting data from system file
ml

,Discovery,Lens Kind,Lens Grade,Einstein_R quality,z_Lens quality,z_Source quality,Description,Einstein_R,Einstein_R_err,z_lens,z_lens_err,z_source,z_source_err,Stellar_v_disp,Stellar_v_disp_err,Discovery Date,Name,N Images
0,SLACS,GAL-GAL,A,SIE model,spectroscopic,spectroscopic,Luminous red galaxy lensing three background o...,1.16,0.02,0.44,0.0001,1.192,0.001,232,39,2008-08-01,SDSS J0008-0004,3
